In [1]:
import re
import codecs
from difflib import SequenceMatcher
from nltk.tag.stanford import StanfordPOSTagger
dirname = '/home/clau/Documents/journal_five_countries/'
import string
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stopwords.append(['mine','interested in', 'likes', 'like', 'love', 'supporter'])

def load_occupations(dirname):
    occupations_file = codecs.open(dirname+'occupations.txt', 'r')
    occupations = list()
    for line in occupations_file:
        cleaned_line = re.sub(r'\d', '', line)
        cleaned_line = cleaned_line.lower()
        cleaned_line = cleaned_line.strip()
        occupations.append(cleaned_line)
    occupations_file.close()
    return occupations

def load_descriptions(dirname):
    descriptions_file = codecs.open(dirname+'audience_descriptions_ireland.txt', 'r')
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    descriptions = list()
    for line in descriptions_file:
        parts = line.split('\t')
        cleaned_line = re.sub(r'^\w\d\s+', '', parts[1])
        cleaned_line = cleaned_line.lower()
        cleaned_line = cleaned_line.strip()
        cleaned_line = cleaned_line.translate(translator)
        descriptions.append(parts[0]+'\t'+cleaned_line+'\t'+parts[2]+'\t'+parts[3]+'\t'+parts[4].strip())
    descriptions_file.close()
    return descriptions

def extract_occupation():
    occupations_list = load_occupations(dirname)
    descriptions_list = load_descriptions(dirname)
    occupations = list()
    for line in descriptions_list:
        found = 0
        parts = line.split('\t')
        for occupation in occupations_list:
            if occupation in parts[1]:
                occupations.append(parts[0]+'\t'+parts[1]+'\t'+parts[2]+'\t'+parts[3]+'\t'+parts[4]+'\t'+occupation)
                found = 1
                break
        if found == 0:
            occupations.append(parts[0]+'\t'+parts[1]+'\t'+parts[2]+'\t'+parts[3]+'\t'+parts[4]+'\t'+'none')
    return occupations

def search_before(text,target,before):
    parts = text.split(' ')
    i = 0
    hobby_before = before
    while i < len(parts):
        if parts[i] == target:
            if i <= len(parts)-2 and parts[i+1] == 'of':
                if parts[i+2] == 'the' or parts[i+2] == 'all':
                    hobby_before = hobby_before +' '+(parts[i]+' '+parts[i+1]+' '+parts[i+2]+' '+parts[i+3])
            else:
                hobby_before = hobby_before +' '+(parts[i-1]+' '+parts[i])
        i+=1
    return hobby_before

def search_after(text,target):
    parts = text.split(' ')
    i = 0
    hobby_after = ''
    while i < len(parts):
        if parts[i] == target:
            if target == 'like':
                if parts[i-1] == 'i':
                    while i < len(parts)-1:
                        hobby_after = hobby_after+' '+parts[i+1]
                        i+=1
                    hobby_after = 'i like'+hobby_after
            elif target == 'dislike':
                if parts[i-1] == 'i':
                    while i < len(parts)-1:
                        hobby_after = hobby_after+' '+parts[i+1]
                        i+=1
                    hobby_after = 'i dislike'+hobby_after
            else:
                hobby_after = parts[i]
                while i < len(parts)-1:
                    hobby_after = hobby_after+' '+parts[i+1]
                    i+=1
        i+=1
    return hobby_after

def neutralize_gender(genderized_line):
    neutral_line = ''
    found = 0
    family = dict()
    family['parent'] = {'mum','dad','daddy','mommy','mom','parent','mother','father','mammy'}
    family['grandparent'] = {'grandmother','grandma','grandpa','grandfather','granny'}
    family['child'] = {' son  ','daughter'}
    family['sibling'] = {'brother','sister'}
    family['person'] = {'lady','woman','man','guy','girl'}
    family['spouse'] = {'wife','husband','married','hubby'}
    family['family oriented'] = {'family guy', 'family oriented'}
    family['the'] = {'his', 'her'}
    for k,v in family.items():
        for v in family[k]:
            if v in genderized_line:
                i=0
                p = genderized_line.split()
                while i < len(p):
                    if v == p[i]:
                        found+=1
                        genderized_line = genderized_line.replace(v,k)
                    i+=1
                if found == 0:
                    neutral_line = genderized_line
    neutral_line = genderized_line
    neutral_line = neutral_line.replace('parent ted', 'father ted')
    if found == 0:
        neutral_line = genderized_line
    return neutral_line

def neutralize_gender_no_hobby(genderized_line):
    neutral_line = ''
    found = 0
    family = dict()
    family['parent'] = {'mum','dad','daddy','mommy','mom','parent','mother','father','mammy'}
    family['grandparent'] = {'grandmother','grandma','grandpa','grandfather','granny'}
    family['child'] = {' son  ','daughter'}
    family['sibling'] = {'brother','sister'}
    family['spouse'] = {'wife','husband','married','hubby'}
    family['family oriented'] = {'family guy', 'family oriented'}
    for k,v in family.items():
        for v in family[k]:
            if v in genderized_line:
                i=0
                found+=1
                p = genderized_line.split()
                while i < len(p):
                    if v == p[i]:
                        if i < len(p)-2 and p[i+1]=='of':
                            neutral_line = neutral_line+' '+p[i].replace(v,k)+' '+p[i+1]+' '+p[i+2]
                        elif i < len(p)-2 and p[i+1]=='to':
                            neutral_line = neutral_line+' '+p[i].replace(v,k)+' '+p[i+1]+' '+p[i+2]
                        else:
                            neutral_line = neutral_line+' '+p[i].replace(v,k)
                    i+=1
                if neutral_line == '':
                    neutral_line = 'none'
    if found == 0:
        neutral_line = 'none'
    return neutral_line

def extract_hobbies_family():
    hobbies = dict()
    line_with_hobby_and_family = list()
    hobbies['before'] = {'addict', 'supporter', 'fan','fanatic','devotee','lover', 'drinker', 'aficionado', 'player','enthusiast','junkie'}
    hobbies['after'] = {'likes', 'like', 'avid', 'compulsive', 'loves', 'interest', 'interests', 'dislikes', 'dislike', 'hates', 'i hate','interested'}
    for line in extract_occupation():
        parts = line.split('\t')
        description = parts[1]
        hobby = ''
        before = ''
        new_before = ''
        after = ''
        for h in hobbies['after']:
            if h in description and len(after)==0:
                description = description.replace('man utd', 'mufc').replace('manchester united','mufc').replace('man united', 'mufc').replace('liverpool fc','lfc')
                after = search_after(description.replace('  ',' '),h)
        for h in hobbies['before']:
            if h in description:
                description = description.replace('man utd', 'mufc').replace('manchester united','mufc').replace('man united', 'mufc').replace('liverpool fc','lfc')
                new_before = search_before(description.replace('  ',' '),h,before)
                before = new_before
        if before in after:
            hobby = hobby+' '+(after.strip())
            hobby = neutralize_gender(hobby)
            hobby = hobby.strip()
        else:
            if len(after.strip()) > 0:
                hobby = hobby.strip()+' '+before.strip()+', '+after.strip()
                hobby = neutralize_gender(hobby)
                hobby = hobby.strip()
            else:
                hobby = hobby.strip()+' '+before.strip()
                hobby = neutralize_gender(hobby)
                hobby = hobby.strip()
        if len(hobby) > 2:
            line_with_hobby_and_family.append(parts[0]+'\t'+parts[5]+'\t'+hobby.replace('  ',' '))
        else:
            family_status = neutralize_gender_no_hobby(description)
            family_status = family_status.strip()
            line_with_hobby_and_family.append(parts[0]+'\t'+parts[5]+'\t'+family_status)
    return line_with_hobby_and_family

In [2]:
for h in extract_hobbies_family():
    print(h)

220725381	none	interested in environment energy frugal innovation
314048832	none	none
3003515385	psephologist	compulsive book buyer happiest at cinema or opera views expressed here are most definitely mine and mine alone
82444899	none	none
315468577	none	parent to six
57148450	journalist	none
2788295746	none	none
22938752	none	lfc supporter gaa supporter
2322466020	none	none
228066656	none	likes cricket science economics parent to 4 spouse to 1 sex positive former bi poly activist known to compromise
2760770915	none	none
2468349552	none	none
2933917781	none	none
18745483	teacher	none
6293752	none	none
476055817	writer	none
763627830	none	news junkie
1171393471	none	none
3303803926	guitarist	interest in criminal law and irish history lead guitarist and celtic fan
1048054902	none	none
1469451456	none	none
15863239	none	none
2244480588	none	none
1467616940	none	none
302911335	consultant	none
119494687	editor	none
20713871	none	none
130159013	author	none
220729806	corr	none
502272875	corr	